In [ ]:
import os, sys, math, time, pickle, gc

import numpy as np

import pandas as pd

from tqdm import tqdm

import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from torch.amp import autocast, GradScaler


# ============================================================================

# 1. CONFIG

# ============================================================================

CONFIG = {

    # --- Input Embeddings ---
    "EMBED_DIR": "/kaggle/input/cafa6-embeds",

    # --- Input Labels & Metadata ---
    "WORK_DIR": "/kaggle/working",
    
    "LABEL_DIR": "/kaggle/input/c99-cafa6",

    'VOCAB_FILE': "vocab_C99_remove.csv",
    'TARGET_FILE': "train_targets_C99.pkl",
    'TRAIN_IDS': "train_ids_C99_split.npy",
    'VAL_IDS': "val_ids_C99_split.npy",
    
    "IA_FILE": "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",

    'TAXON_PKL': "/kaggle/input/cafa6-embeds/taxon_mapping_K140.pkl",

    # --- Model Params ---
    "input_dim": 1280,
    "hidden_dims": [2048, 4096],
    "dropout": 0.3,
    'taxon_embed_dim': 64,
    
    "batch_size": 16,
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "epochs": 25,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

# ============================================================================
# 2. MODEL(CÓ TAXONOMY)
# ============================================================================

class WideProteinMLP_WithTaxon(nn.Module):
    def __init__(self, input_dim, num_classes, num_taxa, taxon_dim=64, hidden_dims=[2048, 4096], dropout=0.3):
        super().__init__()
        
        # 1. Nhánh Protein (ESM-2) 
        self.bn_input = nn.LayerNorm(input_dim)
        
        # 2. Nhánh Taxonomy 
        self.taxon_embedding = nn.Embedding(num_taxa, taxon_dim)
        self.taxon_norm = nn.LayerNorm(taxon_dim)  

        self.unk_idx = num_taxa - 1

        with torch.no_grad():
            self.taxon_embedding.weight[self.unk_idx].zero_()
        
        self.register_buffer(
            "unk_fixed_vector",
            self.taxon_embedding.weight[self.unk_idx].clone()
        )
        
        # 3. Tổng hợp 
        combined_dim = input_dim + taxon_dim
        
        layers = []
        prev = combined_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            prev = h
            
        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x_seq, x_tax):
        
        feat_seq = self.bn_input(x_seq)
        feat_tax = self.taxon_embedding(x_tax)
        feat_tax = self.taxon_norm(feat_tax)  
        
        # [Batch, 1280 + 64]
        combined = torch.cat([feat_seq, feat_tax], dim=1)
        
        return self.net(combined)

# ============================================================================
# 3. DATASET (LOAD TAXON)
# ============================================================================
class CAFA6Dataset(Dataset):
    def __init__(self, ids_file, targets_file, embed_dir, num_classes, taxon_pkl):
        # Load ID & Labels n
        path = os.path.join(CONFIG['LABEL_DIR'], ids_file)
        if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], ids_file)
        self.ids = np.load(path)
        
        t_path = os.path.join(CONFIG['LABEL_DIR'], targets_file)
        if not os.path.exists(t_path): t_path = os.path.join(CONFIG['WORK_DIR'], targets_file)
        with open(t_path, 'rb') as f: self.labels_dict = pickle.load(f)
            
        self.num_classes = num_classes
        self.id_to_embed_idx = {}
        with open(os.path.join(embed_dir, "train_ids.txt"), 'r') as f:
            for idx, line in enumerate(f): self.id_to_embed_idx[line.strip()] = idx
        self.embed_matrix = np.load(os.path.join(embed_dir, "train_embeds.npy"), mmap_mode='r')
        
        # Load Taxon Mapping
        
        if os.path.exists(taxon_pkl):
            tax_path = taxon_pkl
        else:
            tax_path = os.path.join(CONFIG['WORK_DIR'], os.path.basename(taxon_pkl))
            print("⚠️ USING TAXON PKL FROM WORK_DIR:", tax_path)
            
        with open(tax_path, 'rb') as f:
            tax_data = pickle.load(f)
        
        self.prot_to_taxon = tax_data['prot_to_taxon_idx'] 
        self.default_tax = tax_data['num_taxa_classes'] - 1 

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        prot_id = self.ids[idx]
        
        # 1. Embed
        embed_idx = self.id_to_embed_idx.get(prot_id)
        if embed_idx is None:
            feat = torch.zeros(CONFIG["input_dim"], dtype=torch.float32)
        else:
            feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float()
        
        # 2. Target
        target = torch.zeros(self.num_classes, dtype=torch.float)
        indices = self.labels_dict.get(prot_id, [])
        if len(indices) > 0: target[indices] = 1.0
            
        # 3. Taxon ID
        taxon_idx = self.prot_to_taxon.get(prot_id, self.default_tax)
        
        return feat, torch.tensor(taxon_idx, dtype=torch.long), target


# ============================================================================

# 4. LOSS: ASL OPTIMIZED 

# ============================================================================


class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=0, clip=0.05, eps=1e-8):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
    def forward(self, x, y):
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid
        if self.clip > 0: xs_neg = (xs_neg + self.clip).clamp(max=1)
        pt = y * xs_pos + (1 - y) * xs_neg
        log_pt = torch.log(pt.clamp(min=self.eps))
        pos_weight = (1 - xs_pos) ** self.gamma_pos
        neg_weight = (1 - xs_neg) ** self.gamma_neg
        weighted_loss = - (pos_weight * log_pt * y + neg_weight * log_pt * (1-y))
        return weighted_loss.sum() / x.size(0)

# ============================================================================

# 5. METRIC & TRAINING LOOP

# ============================================================================


def calculate_fmax_subset(preds, targets, ia_weights):
    w = ia_weights.reshape(1, -1)
    true_sum = np.sum(targets * w, axis=1)
    valid_mask = true_sum > 0
    if valid_mask.sum() == 0: return 0.0
    p_sub = preds[valid_mask]; t_sub = targets[valid_mask]; w_sub = w; w_true_sub = true_sum[valid_mask]
    best_f1 = 0.0
    thresholds = np.linspace(0.0, 1.0, 51) 
    for tau in thresholds:
        cut = (p_sub >= tau).astype(int)
        tp = np.sum((cut * t_sub) * w_sub, axis=1)
        pred_sum = np.sum(cut * w_sub, axis=1)
        prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
        rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        avg_p = np.mean(prec); avg_r = np.mean(rec)
        if (avg_p + avg_r) > 0: f1 = 2 * avg_p * avg_r / (avg_p + avg_r)
        else: f1 = 0.0
        if f1 > best_f1: best_f1 = f1
    return best_f1

def validate_with_loss(model, loader, vocab_df, ia_weights, device, criterion):
    model.eval()
    all_preds, all_targets = [], []
    val_loss_sum = 0.0
    n_batches = 0

    with torch.no_grad():
        for x_seq, x_tax, y in loader:
            x_seq = x_seq.to(device)
            x_tax = x_tax.to(device)
            y = y.to(device)

            with autocast(device_type="cuda"):
                logits = model(x_seq, x_tax)
                loss = criterion(logits, y)

            val_loss_sum += loss.item()
            n_batches += 1

            all_preds.append(torch.sigmoid(logits).cpu().numpy())
            all_targets.append(y.cpu().numpy())

    val_loss = val_loss_sum / n_batches

    Y_p = np.vstack(all_preds)
    Y_t = np.vstack(all_targets)

    scores = {}
    for aspect in ['MFO', 'BPO', 'CCO']:
        col_indices = vocab_df.index[vocab_df['aspect'] == aspect].tolist()
        if not col_indices:
            continue
        scores[aspect] = calculate_fmax_subset(
            Y_p[:, col_indices], 
            Y_t[:, col_indices], 
            ia_weights[col_indices]
        )

    avg_fmax = np.mean(list(scores.values()))
    return val_loss, avg_fmax, scores


# ============================================================================
# 5. MAIN TRAINING LOOP
# ============================================================================
def train_c99_taxon():
    print("🚀 START TRAINING C95 WITH TAXONOMY (SCRATCH)...")
    
    # 1. Load Resources
    vocab_df = pd.read_csv(os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE']))
    num_classes = len(vocab_df)
    
    try:
        ia_df = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'], header=None)
        ia_map = dict(zip(ia_df.term, ia_df.ia))
        ia_weights = np.array([ia_map.get(t, 1.0) for t in vocab_df.term.values])
    except: ia_weights = np.ones(num_classes)
    
    # 2. Dataset & Model
    tax_pkl_path = CONFIG['TAXON_PKL'] if os.path.exists(CONFIG['TAXON_PKL']) else os.path.join(CONFIG['WORK_DIR'], CONFIG['TAXON_PKL'])
    with open(tax_pkl_path, 'rb') as f: tax_data = pickle.load(f)
    num_taxa = tax_data['num_taxa_classes']
    print(f"   Num Taxa Classes: {num_taxa}")

    train_ds = CAFA6Dataset(CONFIG['TRAIN_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes, CONFIG['TAXON_PKL'])
    val_ds = CAFA6Dataset(CONFIG['VAL_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes, CONFIG['TAXON_PKL'])
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size']*2, shuffle=False, num_workers=2)
    
    # Init Model
    model = WideProteinMLP_WithTaxon(
        input_dim=CONFIG['input_dim'], 
        num_classes=num_classes, 
        num_taxa=num_taxa, 
        taxon_dim=CONFIG['taxon_embed_dim'],
        hidden_dims=CONFIG['hidden_dims'], 
        dropout=CONFIG['dropout']
    ).to(CONFIG['device'])
    
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
    criterion = AsymmetricLossOptimized(gamma_neg=3.0, gamma_pos=1.2, clip=0.05)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CONFIG['lr'], steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
    scaler = GradScaler("cuda")

    best_score = 0.0
    
    for epoch in range(CONFIG['epochs']):
        model.train()
        loss_sum = 0
        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)
        
        for x_seq, x_tax, y in pbar:
            x_seq = x_seq.to(CONFIG['device'])
            x_tax = x_tax.to(CONFIG['device'])
            y = y.to(CONFIG['device'])
            
            optimizer.zero_grad()
            with autocast(device_type="cuda"):
                logits = model(x_seq, x_tax) 
                loss = criterion(logits, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            #  CHẶN UNK HỌC LẠI SAU MỖI BƯỚC UPDATE
            with torch.no_grad():
                emb = model.module.taxon_embedding if hasattr(model, "module") else model.taxon_embedding
                unk_idx = model.module.unk_idx if hasattr(model, "module") else model.unk_idx
                unk_vec = model.module.unk_fixed_vector if hasattr(model, "module") else model.unk_fixed_vector
                emb.weight[unk_idx].copy_(unk_vec)
            
            scheduler.step()
            
            

            loss_sum += loss.item()
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        if epoch < 4 or epoch % 2 == 0:
            val_loss, val_fmax, val_details = validate_with_loss(
                model, val_loader, vocab_df, ia_weights, CONFIG['device'], criterion
            )
        else:
            val_loss = -1
            val_fmax = -1
            val_details = {}
        
        print(
            f"Epoch {epoch+1}: "
            f"Train Loss={loss_sum/len(train_loader):.4f} | "
            f"Val Loss={val_loss:.4f} | "
            f"Val F-max={val_fmax:.4f} {val_details}"
        )
        
        if val_fmax > best_score:
            best_score = val_fmax
            torch.save(model.state_dict(), "best_model_c99_taxon.pth")
            print("   🏆 Saved Best Model (With Taxon)!")

if __name__ == "__main__":
    train_c99_taxon()


🚀 START TRAINING C95 WITH TAXONOMY (SCRATCH)...
   Num Taxa Classes: 141


Ep 1:   0%|          | 0/4636 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1: Train Loss=48.4454 | Val Loss=36.7104 | Val F-max=0.4164 {'MFO': 0.4511074268174924, 'BPO': 0.31310458641950534, 'CCO': 0.4850624851299321}
   🏆 Saved Best Model (With Taxon)!


Epoch 2: Train Loss=34.8172 | Val Loss=33.5167 | Val F-max=0.4672 {'MFO': 0.5152544827787097, 'BPO': 0.34940452879138123, 'CCO': 0.5369393371142823}
   🏆 Saved Best Model (With Taxon)!


Epoch 3: Train Loss=32.6078 | Val Loss=32.5096 | Val F-max=0.4915 {'MFO': 0.5421428515222344, 'BPO': 0.3721202659385291, 'CCO': 0.5601608724794943}
   🏆 Saved Best Model (With Taxon)!


...

Epoch 11: Train Loss=28.8322 | Val Loss=31.0693 | Val F-max=0.5453 {'MFO': 0.6089648974788927, 'BPO': 0.42326259102916397, 'CCO': 0.6035403968267343}
   🏆 Saved Best Model (With Taxon)!


Epoch 12: Train Loss=26.5012 | Val Loss=-1.0000 | Val F-max=-1.0000 {}


Epoch 13: Train Loss=28.1675 | Val Loss=30.7357 | Val F-max=0.5549 {'MFO': 0.6204340760214903, 'BPO': 0.43573848103639556, 'CCO': 0.6084855125742561}
   🏆 Saved Best Model (With Taxon)!


Epoch 14: Train Loss=25.2765 | Val Loss=-1.0000 | Val F-max=-1.0000 {}


Epoch 15: Train Loss=23.3206 | Val Loss=36.1480 | Val F-max=0.5637 {'MFO': 0.6314958137001555, 'BPO': 0.4448282732837725, 'CCO': 0.6146354826207328}
   🏆 Saved Best Model (With Taxon)!
